In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/Users/haoshen/Downloads/responses.csv")
df.drop(columns="Timestamp", inplace=True)
scores_df = df.copy()
scores_df["Score"] = 0
scores_df


,Name,How many sumikkogarashi's are there?,How many eevees are there?,Name Question,Citation Question,Reviews Question,LinkedIn Question,Numbers Question,Party Question,Animal Question,Score
0,NaN,28,28,23,20,800,200,"45,86,86",23,40,0
1,hao,28,29,24,551,44,432,"52,23,45",234,4,0
2,bob,28,28,27,240,80,20,"45,68,86",2,40,0
3,boe,29,29,29,41,4,42,"52,86,49",23,49,0


In [3]:
ANSWERS = {
    "Name": 1,
    "How many sumikkogarashi's are there?": 29,
    'How many eevees are there?': 29,
    'Name Question': 999999,
    'Citation Question': 20,
    'Reviews Question': 382,
    'LinkedIn Question': 794, 
    'Party Question': 999999,
    'Animal Question': 999999,
}
QUESTIONS = list(ANSWERS.keys())
SCORES = []

In [4]:
# for 1 and 2
QUESTION_NUM = 1
QUESTION_TEXT = QUESTIONS[QUESTION_NUM]
scores = (df[QUESTION_TEXT] == ANSWERS[QUESTION_TEXT]).astype(float)
scores_df[QUESTION_TEXT] = scores
scores_df["Score"] += scores
scores_df.sort_values(by=QUESTION_TEXT, ascending=False)[["Name", QUESTION_TEXT]]

,Name,How many sumikkogarashi's are there?
3,boe,1.0
0,NaN,0.0
1,hao,0.0
2,bob,0.0


In [5]:
# for 3 4 5 6 7 8
QUESTION_NUM = 8
QUESTION_TEXT = QUESTIONS[QUESTION_NUM]
abs_value = np.abs(df[QUESTION_TEXT] - ANSWERS[QUESTION_TEXT]).astype(float)
scores = np.sqrt(1/abs_value.rank(method="min", ascending=True))
scores_df[QUESTION_TEXT] = scores
scores_df["Score"] += scores
scores_df.sort_values(by=QUESTION_TEXT, ascending=False)[["Name", QUESTION_TEXT]]

,Name,Animal Question
3,boe,1.000000
0,NaN,0.707107
2,bob,0.707107
1,hao,0.500000


In [14]:
# numbers game
from collections import Counter
from re import split


def unique_number_scoring(series: pd.Series) -> pd.Series:
    """
    Calculates the score for each entry based on the sum of numbers
    that no one else has listed.
    
    Parameters:
    series (pd.Series): A Pandas Series of comma-separated numbers.

    Returns:
    pd.Series: A Series containing the unique-number scores.
    """
    # Convert series to lists of numbers
    split_series = series.str.split(',').apply(lambda x: list(map(int, x)))
    for chosen_numbers in split_series:
        assert len(chosen_numbers) == 3
        assert min(chosen_numbers) > 0
        assert max(chosen_numbers) < 101
    # Flatten all numbers and count occurrences
    number_counts = Counter([num for nums in split_series for num in nums])
    for k in sorted(number_counts)[::-1]:
        print(k, "selected", number_counts[k], "times")
    
    # Compute unique scores for each entry
    scores = split_series.apply(lambda nums: sum(num for num in nums if number_counts[num] == 1))
    
    return scores

scores = unique_number_scoring(df["Numbers Question"])
scores = np.sqrt(1/(scores.rank(method="min", ascending=False))) # the higher the score the better
scores_df["Numbers Question"] = scores
scores_df["Score"] += scores
scores_df.sort_values(by="Numbers Question", ascending=False)[["Name", "Numbers Question"]]


0    [45, 86, 86]
1    [52, 23, 45]
2    [45, 68, 86]
3    [52, 86, 49]
Name: Numbers Question, dtype: object
86 selected 4 times
68 selected 1 times
52 selected 2 times
49 selected 1 times
45 selected 3 times
23 selected 1 times


,Name,Numbers Question
2,bob,1.000000
3,boe,0.707107
1,hao,0.577350
0,NaN,0.500000


In [7]:
scores_df.sort_values(by="Score", ascending=False)[["Name", "Score"]]

,Name,Score
3,boe,2.707107
2,bob,1.707107
0,NaN,1.207107
1,hao,1.077350


In [8]:
# Do a probabilistic version???